# __MINI PROJETO DE ANÁLISE EXPLORATÓRIA DE DADOS, USANDO IMDB COMO BASE DE DADOS__

Neste mini-projeto vamos apresentar um guia básico de análise exploratória de dados usando Linguagem Python, Linguagem SQL e Banco de Dados SQlite.
Usaremos dados reais deponíveis publicamente,dados sobre filmes no IMDB.

Ao aplicarmos análise exploratoria de dados vamos responder a estas 10 perguntas

. 1-Quais são as Categorias de Filmes Mais Comuns no IMDB?

. 2-Qual o Número de Títulos Por Gênero?

. 3-Qual a Mediana de Avaliações dos Filmes por Gênero?

. 4-Qual a Mediana de Avaliações dos Filmes Em Relação ao Ano de Estréia?

. 5-Qual o Número de Filmes Avaliados Por Gênero Em Relação ao Ano de Estréia?

. 6-Qual o Filme Com Maior Tempo de Duração? Calcule os Percentis

. 7-Qual a Relação Entre Duração e Gênero?

. 8-Qual o Número de Filmes Produzidos Por País?

. 9-Quais São os Top 10 Melhores Filmes?

. 10-Quais São os Topo 10 Piores Filmes?

In [ ]:
# Instalando o pacote e base de dados imdb
#!pip install -q imdb-sqlite

In [ ]:
# Instalando o pacote
#!pip install -q pycountry

In [ ]:
# Importando as bibliotecas necessárias para a análise

import re # usado para expressoes regulare(processar textos)
import time # usado par contagem de tempo(calcular o tempo de execução e trabalhar com datas)
import sqlite3 as sql # banco de dados sqlite(pacote para manipular banco de dados sqlite em python)
import pycountry # para manipular os países no banco de dados imdb
import numpy as np # usado par manipulação de dados em python(array)
import pandas as pd # usado par manipulação de dados em python(dados tabulares)
import matplotlib.pyplot as plt # usado para visualização de dados
import seaborn as sns # usado para visualização de dados
from matplotlib import cm # # usado para visualização de dados
from sklearn.feature_extraction.text import CountVectorizer # usado para machine learning em python
import warnings # para filtrar warnings para não poluir o jupyteNotebook
warnings.filterwarnings('ignore')
sns.set_theme(style='whitegrid') # Definindo um estilo(tema) para os gráficos

# __Carregando os dados__

In [ ]:
# Baixando os dados
!imdb-sqlite

In [ ]:
# Conecta no banco de dados

conn = sql.connect('imdb.db')

In [ ]:
# Extrai a lista de tabelas

tabelas = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type='table'", conn)

In [ ]:
# Tipo do objeto
type(tabelas)

In [ ]:
# Visualizando o resultado

tabelas.head()

In [ ]:
# Convertendo o dataFrame em uma lista
tabelas = tabelas['Table_Name'].values.tolist()

In [ ]:
# Vamos percorrer a lista de tabelas no banco de dados e extrair o esquema de cada uma
for tabela in tabelas:
    consulta = f'PRAGMA TABLE_INFO({tabela})'
    resultado = pd.read_sql_query(consulta, conn)
    print('Esquema da tabela:', tabela)
    display(resultado)
    print('-'*100)
    print('\n')


# Fazendo a análise exploratória dos dados

#### 1-__Quais são as Categorias de Filmes Mais Comuns no IMDB?__

Quais são os principais tipos(categorias) dos títulos(filmes)?

In [ ]:
# Criar a consulta SQL
consulta1 = '''SELECT type, COUNT(*) AS COUNT FROM titles GROUP BY type'''

In [ ]:
# Extrai o resultado
resultado1 = pd.read_sql_query(consulta1, conn)

In [ ]:
# Visualizando o resultado
display(resultado1)

In [ ]:
# Calculando o percentual para cada tipo
resultado1['percentual'] = (resultado1['COUNT']/resultado1['COUNT'].sum())*100

In [ ]:
# Visualizando o resultado
display(resultado1)

In [ ]:
# Será criado um gráfico com apenas 4 categorias:
# As 3 categorias com mais títulos e 1 categoria com todo o restante

# Cria um dicionário vazio
others = {}

# Filtra o percentual em 5% e soma o total
others['COUNT'] = resultado1[resultado1['percentual'] < 5]['COUNT'].sum()

# Gravando o percentual
others['percentual'] = resultado1[resultado1['percentual'] < 5]['percentual'].sum()

# Ajusta o nome
others['type'] = 'others'

In [ ]:
# Visualizando 
others

In [ ]:
# Filtrando o dataFrame de outras categorias
resultado1 = resultado1[resultado1['percentual'] > 5]

In [ ]:
# Append com o dataFrame de outras categorias
resultado1 = resultado1.append(others, ignore_index=True)

In [ ]:
# Ordena o resultado
resultado1 = resultado1.sort_values(by='COUNT', ascending=False)

In [ ]:
# Visualizando
resultado1.head()

In [ ]:
# Ajusta os labels
labels = [str(resultado1['type'][i])+' '+'['+str(round(resultado1['percentual'][i],2))+'%'+']' for i in resultado1.index]

In [ ]:
# Fazendo o Plot

# Mapa de cores
# https://matplotlib.org/stable/tutorials/colors/colormaps.html
cs = cm.Set3(np.arange(100))

# Cria a figura
f = plt.figure()
# Pie plot
plt.pie(resultado1['COUNT'], labeldistance=1, radius=3, colors=cs, wedgeprops=dict(width=0.8))
plt.legend(labels=labels, loc='center', prop={'size':12})
plt.title('Distribuição de Títulos', loc='Center', fontdict={'fontsize':20,'fontweight':20})
plt.show()

#### 2-__Qual o Número de Títulos Por Gênero?__

Vamos calcular o número de filmes por gênero e entregar o resultado em valor percentual.

In [ ]:
# Criando a consulta SQL
consulta2 = '''SELECT genres, COUNT(*) FROM titles WHERE type = 'movie' GROUP BY genres'''

In [ ]:
# Resultado 
resultado2 = pd.read_sql_query(consulta2, conn)

In [ ]:
# Visualizando o resulta
display(resultado2)

In [ ]:
# Convertendo as strings para minúsculo
resultado2['genres'] = resultado2['genres'].str.lower().values

In [ ]:
# Removendo valores NA(ausente)
temp = resultado2['genres'].dropna()

Usamos o ___counvertorizer___ para converter a coluna de gêneros em um vetor one-hot encoded para contar o número de filmes em cada gênero

Vamos criar um vetor usando expressão regular para filtrar as strings

In [ ]:
padrao = '(?u)\\b[\\w-]+\\b'
vetor = CountVectorizer(token_pattern=padrao, analyzer='word').fit(temp)

In [ ]:
# verificando o tipo
type(vetor)

In [ ]:
# Aplica a vetorização ao dataset em valores NA
bag_generos = vetor.transform(temp)

In [ ]:
type(bag_generos)

In [ ]:
# Retorna gêneros únicos
generos_unicos = vetor.get_feature_names()

In [ ]:
# Retornando gêneros únicos
generes_unicos = vetor.get_feature_names()

In [ ]:
# Cria o dataFrame de gêneros(Convertendo para dataFrame)
generos = pd.DataFrame(bag_generos.todense(), columns = generos_unicos, index = temp.index)

In [ ]:
# Visualizando
generos.info()

In [ ]:
# Drop da coluna N(removendo a cluna n)
generos = generos.drop(columns = 'n', axis=0)

In [ ]:
# Calcula o percentual
generos_percentual = 100 * pd.Series(generos.sum()).sort_values(ascending = False) / generos.shape[0]

In [ ]:
# Visualizando o resultado
generos_percentual.head(10)

In [ ]:
# Plotando os dados com Plot
plt.figure(figsize=(16, 8))
sns.barplot(x = generos_percentual.values, y = generos_percentual.index, orient = 'h', palette='terrain')
plt.ylabel('Gênero')
plt.xlabel('\nPercentual de Filmes (%)')
plt.title('\nNúmero (Percentual) de Títulos Por Gênero\n')
plt.show();

# 3-Qual a Mediana de Avaliações dos Filmes Por Gênero?

Mediana de avaliações dos filmes

In [ ]:
# Criando a consulta SQL
consulta3 = '''
            SELECT rating, genres FROM
            ratings JOIN titles ON ratings.title_id = titles.title_id
            WHERE premiered <= 2022 AND type = 'movie'
            '''

In [ ]:
# Resulta
resultado3 = pd.read_sql_query(consulta3, conn)

In [ ]:
# Visualizando
display(resultado3)

In [ ]:
# Criando uma função para retornar os gêneros
def retorna_genero(df):
    df['genres'] = df['genres'].str.lower().values
    temp = df['genres'].dropna()
    vetor = CountVectorizer(token_pattern='(?u)\\b[\\w-]+\\b', analyzer='word').fit(temp)
    generos_unicos = vetor.get_feature_names()
    generos_unicos = [genre for genre in generos_unicos if len(genre) > 1]
    return generos_unicos

In [ ]:
# Aplica a função
generos_unicos = retorna_genero(resultado3)

In [ ]:
# Visualizando
generos_unicos

In [ ]:
# Criando listas vazias
genero_counts = [] # contagem de filmes por gênero
genero_ratings = [] # contagem de avaliações

In [ ]:
# Criando loop para adicionar a quantidade dos valores únicos de genero e contagen de suas avaliações
for item in generos_unicos:
    # Retorna a contagem de filmes por gênero
    consulta = 'SELECT COUNT(rating) FROM ratings JOIN titles ON ratings.title_id=titles.title_id WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\''
    resultado = pd.read_sql_query(consulta, conn)
    genero_counts.append(resultado.values[0][0])

    # Retornando a avaliação de filmes por gênero
    consulta = 'SELECT rating FROM ratings JOIN titles ON ratings.title_id=titles.title_id WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\''
    resultado = pd.read_sql_query(consulta, conn)
    genero_ratings.append(np.median(resultado['rating'])) # calculando a mediana da avaliação e adicionando na lista rating

In [ ]:
# Prepara o dataFrame final
df_genero_ratings = pd.DataFrame()
df_genero_ratings['genres'] = generos_unicos
df_genero_ratings['count'] = genero_counts
df_genero_ratings['rating'] = genero_ratings

In [ ]:
# Visualizando 
df_genero_ratings.head(30)

In [ ]:
# Removendo a categiria new, pelo indice
# Não queremos essa informação com gênero

df_genero_ratings = df_genero_ratings.drop(index = 18)

In [ ]:
# Ordenando o resultado
df_genero_ratings = df_genero_ratings.sort_values(by = 'rating', ascending = False)

In [ ]:
# Plotando o resultado
# Plot

# Figure
plt.figure(figsize=(16, 10))

# Barplot
sns.barplot(y=df_genero_ratings.genres, x=df_genero_ratings.rating, orient='h')

# Texto do gráfico
for i in range(len(df_genero_ratings.index)):
    plt.text(4.0,
            i + 0.25,
            str(df_genero_ratings['count'][df_genero_ratings.index[i]]) + ' filmes')

    plt.text(df_genero_ratings.rating[df_genero_ratings.index[i]],
            i + 0.25,
            round(df_genero_ratings['rating'][df_genero_ratings.index[i]],2))

plt.ylabel('Gênero')
plt.xlabel('Mediana da Avaliação')
plt.title('\nMediana de Avaliação Por Gênero\n')
plt.show()

# __4-Qual a Mediana de Avaliação de Filmes Em Relação ao Ano de Estréia?__

Mediana de avaliação dos filmes

In [ ]:
# Fazendo a consulta SQL
consulta4 = '''
            SELECT rating AS Rating,premiered FROM
            ratings JOIN titles ON ratings.title_id = titles.title_id
            WHERE premiered <= 2022 AND type = 'movie'
            ORDER BY premiered
            '''

In [ ]:
# Resultado
resultado4 = pd.read_sql_query(consulta4, conn)

In [ ]:
# Mostrando o resultado
display(resultado4)

In [ ]:
# Calculando a mediana ao longo do tempo (anos)
ratings = []
# fazendo um loop e percorrendo dentro da lista ano de estreia(premiered), calculando a mediana para ano
for year in set(resultado4['premiered']):
    ratings.append(np.median(resultado4[resultado4['premiered'] == year]['Rating']))

In [ ]:
# Mostrando o tipo da variável ratings
type(ratings)

In [ ]:
ratings[1:10]

In [ ]:
# Lista de anos
anos = list(set(resultado4['premiered']))

In [ ]:
anos[1:10]

In [ ]:
# Fazendo o plot no gráfico
plt.figure(figsize = (16,8))
plt.plot(anos, ratings)
plt.xlabel('\nAno')
plt.ylabel('Mediana de Avaliação')
plt.title('\nMediana de Avaliação dos Filmes Em Relação ao Ano de Estréia\n')
plt.show()


# __5-Qual o Número de Filmes Avaliados Por Gênero Em Relação ao Ano de Estréia?__

Número de filmes por gênero. Vamos listar os Top 5.

In [ ]:
# Criando a consulta no banco de dados SQL
consulta5 = '''SELECT genres FROM titles'''

In [ ]:
# Resultado
resultado5 = pd.read_sql_query(consulta5, conn)

In [ ]:
display(resultado5)

In [ ]:
# Retorna gêneros únicos
generos_unicos = retorna_genero(resultado5)

In [ ]:
# Visualiza o resultado
generos_unicos

In [ ]:
# Fazendo a contagem
genero_count = []
for item in generos_unicos:
    consulta = 'SELECT COUNT(*) COUNT FROM titles WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\' AND premiered <=2022'
    resultado = pd.read_sql_query(consulta, conn)
    genero_count.append(resultado['COUNT'].values[0])

In [ ]:
# Preparando o dataFrame
df_genero_count = pd.DataFrame()
df_genero_count['genre'] = generos_unicos
df_genero_count['Count'] = genero_count

In [ ]:
# Calcula os top 5
df_genero_count = df_genero_count[df_genero_count['genre'] != 'n']
df_genero_count = df_genero_count.sort_values(by='Count', ascending=False)
top_generos = df_genero_count.head()['genre'].values

In [ ]:
# Gerando o Plot dos dados

# Figura
plt.figure(figsize=(16,8))

# Loop e Plot
for item in top_generos:
    consulta = 'SELECT COUNT(*) Number_of_movies, premiered Year FROM titles WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\' AND premiered <=2022'
    resultado = pd.read_sql_query(consulta, conn)
    plt.plot(resultado['Year'], resultado['Number_of_movies'])

plt.xlabel('\nAno')
plt.ylabel('Número de Filmes Avaliados')
plt.title('\nNúmeros de Fimes Avaliados Por Gênero Em Relação ao Ano de Estréia\n')
plt.legend(labels = top_generos)
plt.show()

# 6-Qual o Filme Com Maior Tempo de Duração? Calcule os Percentis.

Percentis e filme mais longo da história

In [ ]:
# Consulta SQL
consulta6 = '''
            SELECT runtime_minutes Runtime
            FROM titles
            WHERE type = 'movie' AND Runtime != 'NaN'
            '''

In [ ]:
# Resultado
resultado6 = pd.read_sql_query(consulta6, conn)

In [ ]:
# Visualizando
display(resultado6)

In [ ]:
# Loop para calculo dos percentis
for i in range(101):
    val = i
    percentil = round(np.percentile(resultado6['Runtime'].values, val),2)
    print(f'{val} percentil da duração(runtime) é: {percentil}')

In [ ]:
# Refazendo a consulta e retornando o filme com maior duração
consulta6 = '''
            SELECT runtime_minutes Runtime, primary_title
            FROM titles
            WHERE type = 'movie' AND Runtime != 'NaN'
            ORDER BY Runtime DESC
            LIMIT 1
            '''

In [ ]:
resultado6 = pd.read_sql_query(consulta6, conn)

In [ ]:
display(resultado6)

'Logistics' é o filme mais longo da história(até o momento), com 857 horas(51420 minutos). Dirigido por Daniel Andersson e Erika Magnusson, o filme acompanha todo o ciclo de vida de um pedômetro(dispositivo de rastramento de passos) em ordem cronológica reversa.

# __7-Qual a Relação Entre Duração e Gênero?__
Relação entre duração do filme e gênero

In [ ]:
# Consulta SQL
consulta7 = '''
            SELECT AVG(runtime_minutes) Runtime, genres
            FROM titles
            WHERE type = 'movie'
            AND runtime_minutes != 'NaN'
            GROUP BY genres
            '''

In [ ]:
# Resultado
resultado7 = pd.read_sql_query(consulta7, conn)

In [ ]:
# Retorna gêneros únicos
generos_unicos = retorna_genero(resultado7)

In [ ]:
# Visualizadno
generes_unicos

In [ ]:
# Calculando a duração por gêneros
genero_runtime = []
for item in generos_unicos:
    consulta = 'SELECT runtime_minutes Runtime FROM titles WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\' AND Runtime!=\'NaN\''
    resultado = pd.read_sql_query(consulta, conn)
    genero_runtime.append(np.median(resultado['Runtime']))

In [ ]:
# Prepara o dataFrame
df_genero_runtime = pd.DataFrame()
df_genero_runtime['genre'] = generos_unicos
df_genero_runtime['runtime'] = genero_runtime

In [ ]:
# Removendo índice 18(news)
df_genero_runtime = df_genero_runtime.drop(index = 18)

In [ ]:
# Ordena os dados
df_genero_runtime = df_genero_runtime.sort_values(by = 'runtime', ascending = False)

In [ ]:
# Plot

# Figura
plt.figure(figsize = (16, 8))

# BarPlot
sns.barplot(y = df_genero_runtime.genre, x = df_genero_runtime.runtime, orient = 'h')

# Loop
for i in range(len(df_genero_runtime.index)):
    plt.text(df_genero_runtime.runtime[df_genero_runtime.index[i]], i + 0.25,
        round(df_genero_runtime['runtime'] [df_genero_runtime.index[i]],2))

plt.ylabel('Gênero')
plt.xlabel('\nMediana de Tempo de Duração (Minutos)')
plt.title('\nRelação Entre Duração e Gênero\n')
plt.show()

# __8-Qual o Número de Filmes Produzidos Por País?__
Número de filmes produzidos por país

In [ ]:
# Fazendo a consulta no banco de dados SQL
consulta8 = '''
            SELECT region, COUNT(*) Number_of_movies FROM
            akas JOIN titles ON
            akas.title_id = titles.title_id
            WHERE region != 'None'
            AND type = \'movies\'
            GROUP BY region
            '''

In [ ]:
# Resultado
resultado8 = pd.read_sql_query(consulta8, conn)

In [ ]:
# Visualizando o resultado
resultado8

In [ ]:
# shape
resultado8.shape

In [ ]:
# Número de linhas
resultado8.shape[0]

In [ ]:
# Listas auxiliares
nomes_paises = []
contagem = []

In [ ]:
# Loop para obter países de acordo com a rigião
for i in range(resultado8.shape[0]):
    try:
        coun = resultado8['region'].values[i]
        nomes_paises.append(pycountry.countries.get(alpha_2 = coun).name)
        contagem.append(resultado8['Number_of_movies'].values[i])
    except:
        continue

In [ ]:
# Prepara o DataFrame
df_filmes_paises = pd.DataFrame()
df_filmes_paises['country'] = nomes_paises
df_filmes_paises['Movie_Count'] = contagem

In [ ]:
# Ordena o resultado
df_filmes_paises = df_filmes_paises.sort_values(by='Movie_Count', ascending=False)

In [ ]:
# Visualizando o resultado
df_filmes_paises.head(10)

In [ ]:
# Plot

# Figura
plt.figure(figsize=(20, 8))

# Loop
for i in range(0, 20):
    plt.text(df_filmes_paises.Movie_Count[df_filmes_paises.index[i]]-1,
            i + 0.30,
            round(df_filmes_paises['Movie_Count'][df_filmes_paises.index[i]],2))

plt.ylabel('País')
plt.xlabel('\nNúmero de Filmes')
plt.title('\nNúmero de Filmes Produzidos Por País\n')
plt.show()


# __9-Quais São os Top 10 Melhores Filmes?__
Top 10 fimes com melhor avaliação e mais de 25 mil votos

In [ ]:
# Consulta
consulta9 = '''
            SELECT primary_title AS Movie_Name, genres, rating
            FROM
            titles JOIN ratings
            ON titles.title_id = ratings.title_id
            WHERE titles.type = 'movie' AND ratings.votes >= 25000
            ORDER BY rating DESC
            LIMIT 10
            '''

In [ ]:
# Resultado
top10_melhors_filmes = pd.read_sql_query(consulta9, conn)

In [ ]:
# Visualizando o resultado
display(top10_melhors_filmes)

# __10-Quais São os Top 10 Piores Filmes?__
Top 10 filmes com pior avaliação e mais de 25 mil votos.

In [ ]:
# Consulta
consulta10 = '''
            SELECT primary_title AS Movie_Name, genres, rating
            FROM
            titles JOIN ratings
            ON titles.title_id = ratings.title_id
            WHERE titles.type = 'movie' AND ratings.votes >= 25000
            ORDER BY rating ASC
            LIMIT 10
            '''

In [ ]:
# Resultado
top10_piores_filmes = pd.read_sql_query(consulta10, conn)

In [ ]:
# Mostrando o resultado
display(top10_piores_filmes)

# __FIM DA ANÁLISE DO MINI-PROJETO__